# Retriever 비교 분석 노트북
## 1. 환경 설정 및 필요 라이브러리 설치

In [1]:
# 필요한 패키지 설치
# pip install langchain langchain_community langchain_openai langchain_chroma unstructured matplotlib pandas


import os
import time
import warnings
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
from IPython.display import Markdown, display

api_key = os.getenv("OPENAI_API_KEY")


import sys
sys.path
from langchain_community.document_loaders import UnstructuredURLLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.retrievers import ContextualCompressionRetriever, SelfQueryRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain.retrievers import MultiQueryRetriever
from langchain.schema import Document

ModuleNotFoundError: No module named 'matplotlib'

## 2. 리트리버 이론적 배경
### Similarity Search 작동 원리:

1. 기본 알고리즘:

문서와 쿼리를 동일한 벡터 공간에 임베딩
벡터 간 코사인 유사도 계산: similarity(q, d) = cos(θ) = (q·d)/(||q||·||d||)
유사도가 높은 상위 k개 문서 반환

2. 수식:

코사인 유사도: cos(θ)=q⃗⋅d⃗∣∣q⃗∣∣⋅∣∣d⃗∣∣cos(\theta) = \frac{\vec{q} \cdot \vec{d}}{||\vec{q}|| \cdot ||\vec{d}||}
cos(θ)=∣∣q​∣∣⋅∣∣d∣∣q​⋅d​
여기서 q는 쿼리 벡터, d는 문서 벡터

3. 단계:
쿼리와 문서를 벡터로 변환 (임베딩)
벡터 간 코사인 유사도 계산
유사도 기준으로 정렬
상위 k개 문서 반환


### MMR(Maximal Marginal Relevance) 작동 원리:

1. 기본 알고리즘:

관련성(relevance)과 다양성(diversity) 사이의 균형을 맞추는 기법
이미 선택된 문서와 비슷한 문서를 페널티화하여 다양한 결과 제공


2. 수식:

MMR=argmax⁡di∈R∖S[λ⋅sim1(di,q)−(1−λ)⋅max⁡dj∈Ssim2(di,dj)]MMR = \underset{d_i \in R \setminus S}{\operatorname{argmax}} [\lambda \cdot \text{sim}_1(d_i, q) - (1 - \lambda) \cdot \underset{d_j \in S}{\max} \text{sim}_2(d_i, d_j)]
MMR=di​∈R∖Sargmax​[λ⋅sim1​(di​,q)−(1−λ)⋅dj​∈Smax​sim2​(di​,dj​)]
여기서:

λ: 관련성과 다양성 간의 균형을 조절하는 파라미터 (0~1)
sim₁(d, q): 문서 d와 쿼리 q 사이의 유사도
sim₂(d, d'): 문서 d와 이미 선택된 문서 d' 사이의 유사도
S: 이미 선택된 문서 집합
R: 아직 선택되지 않은 문서 집합




3. 단계:

빈 결과 집합 S로 시작
관련성이 가장 높은 문서를 S에 추가
다음 문서 선택 시 관련성과 다양성을 모두 고려
원하는 수의 문서를 찾을 때까지 반복



### Document Compression Retriever 작동 원리:

1. 기본 알고리즘:

기존 리트리버로 검색한 문서에서 쿼리와 관련된 부분만 추출하는 방식
원본 문서의 관련 내용만을 압축하여 제공


2. 수식:

압축 함수: compress(d,q)→d′compress(d, q) \rightarrow d'
compress(d,q)→d′
압축 비율: ratio=∑∣d′∣∑∣d∣ratio = \frac{\sum|d'|}{\sum|d|}
ratio=∑∣d∣∑∣d′∣​ (여기서 |d|는 문서 d의 길이)



3. 단계:

기본 리트리버를 사용하여 초기 문서 집합 검색
각 문서에 대해 LLM을 사용하여 쿼리에 관련된 정보만 추출
압축된 문서 집합 반환



### Self-Query Retriever 작동 원리:

1. 기본 알고리즘:

자연어 쿼리를 구조화된 쿼리와 필터로 분해
메타데이터 필터링을 사용한 하이브리드 검색 수행


2. 수식:

파싱 함수: parse(q)→(q′,f)parse(q) \rightarrow (q', f)
parse(q)→(q′,f)
필터링: results={d∈candidates∣f(d)=True}results = \{d \in candidates | f(d) = True\}
results={d∈candidates∣f(d)=True}


3. 단계:

LLM을 사용하여 사용자 쿼리를 구조화된 쿼리와 필터로 변환
구조화된 쿼리로 벡터 검색 수행
필터를 적용하여 결과 정제
필터링된 결과 반환



## 3. 데이터 로드 및 처리

In [ ]:
# 테스트에 사용할 URL 데이터 정의
urls = [
    "https://en.wikipedia.org/wiki/Artificial_intelligence",
    "https://en.wikipedia.org/wiki/Machine_learning",
    "https://en.wikipedia.org/wiki/Natural_language_processing",
    "https://en.wikipedia.org/wiki/Computer_vision",
    "https://en.wikipedia.org/wiki/Deep_learning",
    "https://en.wikipedia.org/wiki/Neural_network",
    "https://en.wikipedia.org/wiki/Reinforcement_learning",
    "https://en.wikipedia.org/wiki/Data_science",
    "https://en.wikipedia.org/wiki/Big_data",
    "https://en.wikipedia.org/wiki/Internet_of_things"
]

# 문서 로드 함수
def load_documents():
    print("문서 로드 중...")
    try:
        loader = UnstructuredURLLoader(urls=urls)
        documents = loader.load()
        print(f"총 {len(documents)}개의 문서가 로드되었습니다.")
        
        # 문서 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        chunks = text_splitter.split_documents(documents)
        print(f"문서가 {len(chunks)}개의 청크로 분할되었습니다.")
        
        return chunks
    except Exception as e:
        print(f"문서 로드 중 오류 발생: {e}")
        
        # 실패 시 샘플 데이터 생성
        print("샘플 데이터를 대신 생성합니다.")
        return create_sample_documents()

# URL 로드 실패 시 샘플 데이터 생성 (데모 목적)
def create_sample_documents():
    sample_texts = [
        "인공지능(AI)은 인간의 학습, 문제 해결, 패턴 인식 능력을 모방하는 시스템을 말합니다. 딥러닝, 머신러닝, 자연어 처리 등의 기술이 포함됩니다.",
        "머신러닝은 명시적 프로그래밍 없이 컴퓨터가 데이터에서 학습할 수 있게 하는 AI의 한 분야입니다. 지도학습, 비지도학습, 강화학습으로 나뉩니다.",
        "자연어 처리(NLP)는 컴퓨터가 인간 언어를 이해, 해석, 생성할 수 있도록 하는 기술입니다. 기계 번역, 감정 분석, 챗봇 등에 사용됩니다.",
        "컴퓨터 비전은 컴퓨터가 디지털 이미지와 비디오를 이해할 수 있게 하는 분야입니다. 이미지 인식, 객체 감지, 얼굴 인식 등이 포함됩니다.",
        "딥러닝은 인공 신경망을 사용하여 데이터에서 표현을 학습하는 머신러닝의 하위 분야입니다. 이미지와 음성 인식에서 뛰어난 성능을 보입니다.",
        "신경망은 인간 두뇌의 신경 구조를 모방한 알고리즘으로, 입력층, 은닉층, 출력층으로 구성됩니다. 딥러닝은 깊은(다중) 은닉층을 가진 신경망을 사용합니다.",
        "강화학습은 에이전트가 환경과 상호작용하면서 보상을 최대화하는 행동을 학습하는 머신러닝 방법입니다. 로봇 공학과 게임 AI에 많이 사용됩니다.",
        "데이터 사이언스는 통계, 머신러닝, 도메인 지식을 결합하여 데이터에서 인사이트를 추출하는 학제간 분야입니다. 비즈니스 의사결정에 중요합니다.",
        "빅데이터는 기존 도구로 처리하기 어려운 대규모 데이터셋을 말합니다. 빅데이터는 양(Volume), 속도(Velocity), 다양성(Variety)의 3V로 특징지을 수 있습니다.",
        "사물인터넷(IoT)은 인터넷을 통해 데이터를 수집하고 교환할 수 있는 연결된 장치의 네트워크입니다. 스마트홈, 웨어러블, 스마트시티에 적용됩니다."
    ]
    
    docs = []
    for i, text in enumerate(sample_texts):
        metadata = {"source": f"sample_{i}", "topic": urls[i].split('/')[-1].replace('_', ' ')}
        docs.append(Document(page_content=text, metadata=metadata))
    
    return docs

# 문서 로드
documents = load_documents()

## 4. 벡터 스토어 설정

In [ ]:
# 벡터 스토어 설정 함수
def setup_vectorstore(documents):
    print("벡터 스토어 설정 중...")
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(
        documents=documents, 
        embedding=embeddings,
        collection_name="retriever_comparison"
    )
    print("벡터 스토어가 설정되었습니다.")
    return vectorstore

# 벡터 스토어 설정
vectorstore = setup_vectorstore(documents)

## 5. 리트리버 설정

In [ ]:
# LLM 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 리트리버 설정 함수
def setup_retrievers(vectorstore, llm):
    print("리트리버 설정 중...")
    
    # 1. 기본 유사도 검색
    similarity_retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    
    # 2. MMR (다양성 기반)
    mmr_retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.5}
    )
    
    # 3. 문서 압축 리트리버
    compressor = LLMChainExtractor.from_llm(llm)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=similarity_retriever
    )
    
    # 4. 다중 쿼리 리트리버 (Self-Query 대체)
    multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever=similarity_retriever,
        llm=llm
    )
    
    retrievers = {
        "Similarity Search": similarity_retriever,
        "MMR": mmr_retriever,
        "Document Compression": compression_retriever,
        "Multi-Query (Self-Query 대체)": multi_query_retriever
    }
    
    print("리트리버 설정이 완료되었습니다.")
    return retrievers

# 리트리버 설정
retrievers = setup_retrievers(vectorstore, llm)

## 6. 테스트 쿼리 정의

In [ ]:
# 다양한 유형의 테스트 쿼리 정의
test_queries = [
    "인공지능과 머신러닝의 차이점은 무엇인가요?",
    "자연어 처리 기술의 최신 응용 사례를 알려주세요",
    "딥러닝은 어떻게 작동하나요?",
    "인공지능의 윤리적 문제에는 어떤 것들이 있나요?",
    "빅데이터와 인공지능의 관계에 대해 설명해주세요"
]

## 7. 리트리버 평가 함수

In [ ]:
# 리트리버 평가 함수
def evaluate_retrievers(retrievers, queries):
    results = {}
    timing = {}
    doc_lengths = {}
    
    print("리트리버 평가 시작...")
    
    for name, retriever in retrievers.items():
        print(f"\n{name} 리트리버 평가 중...")
        results[name] = []
        timing[name] = []
        doc_lengths[name] = []
        
        for i, query in enumerate(queries):
            print(f"  쿼리 {i+1}/{len(queries)}: '{query[:30]}...' 처리 중")
            
            start_time = time.time()
            try:
                docs = retriever.get_relevant_documents(query)
                end_time = time.time()
                
                # 결과 저장
                results[name].append(docs)
                timing[name].append(end_time - start_time)
                
                # 문서 길이 저장 (압축 리트리버 평가용)
                total_length = sum(len(doc.page_content) for doc in docs)
                doc_lengths[name].append(total_length)
                
                print(f"    {len(docs)}개 문서 검색 완료, 소요 시간: {timing[name][-1]:.4f}초")
            except Exception as e:
                print(f"    오류 발생: {e}")
                results[name].append([])
                timing[name].append(0)
                doc_lengths[name].append(0)
    
    print("\n리트리버 평가가 완료되었습니다.")
    return results, timing, doc_lengths

# 리트리버 평가 실행
results, timing, doc_lengths = evaluate_retrievers(retrievers, test_queries)

## 9. 리트리버 결과 내용 비교 분석

In [ ]:
# 결과 내용 분석 함수
def analyze_content(results, queries):
    for query_idx, query in enumerate(queries):
        print(f"\n\n=== 쿼리: {query} ===\n")
        
        for name, result_set in results.items():
            if query_idx < len(result_set) and result_set[query_idx]:
                docs = result_set[query_idx]
                print(f"\n--- {name} 리트리버 (상위 1개 결과) ---")
                print(f"문서 내용: {docs[0].page_content[:300]}...")
                
                if hasattr(docs[0], 'metadata') and docs[0].metadata:
                    print(f"메타데이터: {docs[0].metadata}")
            else:
                print(f"\n--- {name} 리트리버 ---")
                print("결과 없음")

# 내용 분석 실행
analyze_content(results, test_queries)

## 10. 리트리버 비교 종합 분석
1. Similarity Search
장점:

구현이 단순하고 직관적
계산 복잡성이 낮아 빠른 검색 가능
대부분의 벡터 DB와 호환성 높음

단점:

결과의 다양성이 부족할 수 있음
쿼리에 있는 키워드와 정확히 일치하는 결과만 반환
의미적 유사성만 고려하고 문맥을 완전히 이해하지 못함

적합한 시나리오:

간단하고 명확한 질문에 대한 빠른 답변 필요시
문서 집합이 작고 잘 구조화된 경우
계산 리소스가 제한적인 환경

2. MMR (Maximal Marginal Relevance)
장점:

다양한 주제의 결과를 제공하여 정보 다양성 확보
중복 정보 감소로 더 포괄적인 결과 제공
λ 파라미터로 다양성과 관련성 사이의 균형 조절 가능

단점:

Similarity Search보다 계산 복잡성이 높음
최적의 λ 값 설정이 어려울 수 있음
다양성을 위해 관련성이 낮은 문서가 포함될 수 있음

적합한 시나리오:

탐색적 검색 시 다양한 관점 필요
광범위한 질문에 대한 종합적 개요 제공
사용자가 주제에 대한 다각적 정보를 원할 때

3. Document Compression Retriever
장점:

불필요한 정보를 제거하여 관련성 높은 내용에 집중
LLM을 활용한 의미적 압축으로 정확도 향상
문서 길이가 길거나 복잡한 경우 효과적

단점:

LLM 호출이 필요하여 시간과 비용 증가
압축 과정에서 중요한 정보가 손실될 가능성
추가적인 처리 단계로 인한 지연

적합한 시나리오:

긴 문서에서 특정 정보 추출 필요시
높은 정확도가 요구되는 질문-답변 시스템
문서 내용이 복잡하고 노이즈가 많은 경우

4. Self-Query Retriever / Multi-Query Retriever
장점:

복잡한 쿼리를 구조화된 형태로 분해 가능
메타데이터 필터링으로 검색 정확도 향상
자연어 쿼리의 의도를 더 잘 파악

단점:

구현이 복잡하고 추가 LLM 호출 필요
메타데이터 구조화가 필요
다른 방식보다 처리 시간이 길 수 있음

적합한 시나리오:

메타데이터가 풍부한 문서 컬렉션
복잡한 조건을 포함한 쿼리 처리
특정 속성이나 카테고리로 결과 필터링 필요시